In [4]:
!pip install lark

In [5]:
from utils1.retreiver_setting import faiss_retriever_loading

In [7]:
import lark

In [9]:
account_retriever, business_retriever, business_retriever2, self_retriever = faiss_retriever_loading()

ImportError: Cannot import lark, please install it with 'pip install lark'.

In [3]:
queries_and_answers = [
    ("삼성바이오로직스 2024년 연결 당기순이익", ["삼성바이오로직스", "2024", "연결", "당기순이익"]),
    ("2024년 LG전자 연구개발 활동", ["2024", "LG전자", "연구개발"]),
    ("2023년 카카오 주요사항 요약", ["2023", "카카오", "주요사항"]),
    ("NAVER 2024년 EPS는 얼마야?", ["NAVER", "2024", "주당순이익"]),
    ("SK하이닉스 2023년 배당금 정보 알려줘", ["SK하이닉스", "2023", "배당금"]),
    ("현대차 2024년 주당액면가액은?", ["현대차", "2024", "주당액면가액"]),
    ("LG화학의 2022년 현금배당 내역", ["LG화학", "2022", "현금배당"]),
    ("카카오 2024년 연구개발 투자 비율은?", ["카카오", "2024", "연구개발", "투자", "비율"]),
    ("삼성전자 2023년 주요사항 정리해줘", ["삼성전자", "2023", "주요사항"]),
    ("NAVER 2024년 연결 실적", ["NAVER", "2024", "연결"]),
    ("LG에너지솔루션의 2023년 EPS", ["LG에너지솔루션", "2023", "주당순이익"]),
    ("SK이노베이션 2024년 주식배당 수익률", ["SK이노베이션", "2024", "주식배당수익률"]),
    ("2024년 현대모비스의 연구개발 성과", ["현대모비스", "2024", "연구개발", "성과"]),
    ("카카오 2022년 별도 재무성과", ["카카오", "2022", "별도"]),
    ("삼성바이오로직스의 2024년 R&D 투자", ["삼성바이오로직스", "2024", "연구개발", "투자"]),
]


In [4]:
from difflib import SequenceMatcher

def evaluate_similarity_score(retriever, queries_and_answers):
    scores = []
    for query, expected_keywords in queries_and_answers:
        docs = retriever.invoke(query)
        all_text = " ".join([doc.page_content for doc in docs])
        score = sum(SequenceMatcher(None, all_text, kw).ratio() for kw in expected_keywords)
        scores.append(score / len(expected_keywords))  # 평균 유사도
    return sum(scores) / len(scores)  # 전체 평균


In [5]:
def evaluate_contains_score(retriever, queries_and_answers):
    scores = []
    for query, expected_keywords in queries_and_answers:
        docs = retriever.invoke(query)
        all_text = " ".join([doc.page_content for doc in docs])
        hit_count = sum(1 for kw in expected_keywords if kw in all_text)
        scores.append(hit_count / len(expected_keywords))  # 포함된 키워드 비율
    return sum(scores) / len(scores)  # 전체 평균


In [6]:
score1_sim = evaluate_similarity_score(business_retriever2, queries_and_answers)
score1_contain = evaluate_contains_score(business_retriever2, queries_and_answers)

score2_sim = evaluate_similarity_score(self_retriever, queries_and_answers)
score2_contain = evaluate_contains_score(self_retriever, queries_and_answers)

print(f"📊 [No Metadata 필터]")
print(f" - 유사도 평가 점수: {score1_sim:.3f}")
print(f" - 키워드 포함 점수: {score1_contain:.3f}")

print(f"📊 [With Metadata 필터]")
print(f" - 유사도 평가 점수: {score2_sim:.3f}")
print(f" - 키워드 포함 점수: {score2_contain:.3f}")

📊 [No Metadata 필터]
 - 유사도 평가 점수: 0.004
 - 키워드 포함 점수: 0.764
📊 [With Metadata 필터]
 - 유사도 평가 점수: 0.004
 - 키워드 포함 점수: 0.644
